In [1]:
import os
import tqdm
import json
import pickle

from IPython.display import display, HTML

!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util

  Using cached sentence-transformers-2.1.0.tar.gz (78 kB)
  Using cached transformers-4.11.3-py3-none-any.whl (2.9 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-win_amd64.whl (2.0 MB)
  Using cached torchvision-0.10.1-cp38-cp38-win_amd64.whl (936 kB)
  Using cached scikit_learn-1.0-cp38-cp38-win_amd64.whl (7.2 MB)
  Using cached scipy-1.7.1-cp38-cp38-win_amd64.whl (33.7 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-win_amd64.whl (1.1 MB)
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Using cached sacremoses-0.0.46-py3-none-any.whl (895 kB)
  Using cached threadpoolctl-3.0.0-py3-none-any.whl (14 kB)
Using legacy 'setup.py install' for sentence-transformers, since package 'wheel' is not installed.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\python38\\Scripts\\sacremoses.exe' -> 'c:\\python38\\Scripts\\sacremoses.exe.deleteme'

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


ModuleNotFoundError: No module named 'sentence_transformers'

In [2]:
input_file = "../input/arxiv/arxiv-metadata-oai-snapshot.json"
data  = []
i = 0
with tqdm.tqdm(total=os.path.getsize(input_file)) as pbar:
     with open(input_file, 'r') as f:
            
        for line in f:
            pbar.update(len(line))
            data.append(json.loads(line))
            i += 1
            if i == 100000:
                break

  5%|▍         | 147937477/3140261595 [00:03<01:20, 37278937.28it/s]


In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

papers = pd.DataFrame(data)
n = len(papers)

ModuleNotFoundError: No module named 'pandas'

In [4]:
papers.tail()

NameError: name 'papers' is not defined

# Generating text for embeddings

In [3]:
paper_texts = [None for i in range(n)]

for i in range(n):
    paper_texts[i] = papers.iloc[i, 3] + '[SEP]' + papers.iloc[i, 10]

NameError: name 'n' is not defined

In [2]:
papers['Input_text'] = paper_texts
papers.head()

NameError: name 'paper_texts' is not defined

In [1]:
print(paper_texts)

NameError: name 'paper_texts' is not defined

# Defining the allenAI Specter Model

In [7]:
model = SentenceTransformer('allenai-specter')

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/622 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/462k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Generating embeddings

In [8]:
corpus_embeddings = model.encode(paper_texts, convert_to_tensor=True)

Batches:   0%|          | 0/3125 [00:00<?, ?it/s]

# Saving the embeddings

In [11]:
with open('embeddings_100k', 'wb') as f:
    pickle.dump(corpus_embeddings, f)

# Defining search paper function

In [60]:
def search_papers(title, abstract):
    query_embedding = model.encode(title+'[SEP]'+abstract, convert_to_tensor=True)

    search_hits = util.semantic_search(query_embedding, corpus_embeddings)
    search_hits = search_hits[0]  #Get the hits for the first query

#   print("\n\nPaper:", title)
#   print("Most similar papers:")
#   for hit in search_hits:
#         try:
#             related_paper = papers[hit['corpus_id']]
#             print("{:.2f}\t{}\t{} {}".format(hit['score'], related_paper['title'], related_paper['venue'], related_paper['year']))
#         except(KeyError):
#             print("keyerror")
    search_hits = pd.DataFrame(search_hits)
#     print(search_hits)
    final_result = pd.DataFrame()
    title = list()
    authors = list()
    arxiv_id = list()
    score = list()
    for i in range(len(search_hits)):
        num = search_hits.iloc[i, 0]
        title.append(papers.iloc[num, 3])
        authors.append(papers.iloc[num, 2])
        arxiv_id.append(papers.iloc[num, 0])
        score.append(search_hits.iloc[i, 1])
        
    final_result['title'] = title
    final_result['authors'] = authors
    final_result['arxiv_id'] = arxiv_id
    final_result['score'] = score
    
    display(HTML(final_result.to_html()))

# Testing

In [61]:
# This paper was the EMNLP 2020 Best Paper
search_papers(title='Digital Voicing of Silent Speech',
              abstract='In this paper, we consider the task of digitally voicing silent speech, where silently mouthed words are converted to audible speech based on electromyography (EMG) sensor measurements that capture muscle impulses. While prior work has focused on training speech synthesis models from EMG collected during vocalized speech, we are the first to train from EMG collected during silently articulated speech. We introduce a method of training on silent EMG by transferring audio targets from vocalized to silent signals. Our method greatly improves intelligibility of audio generated from silent EMG compared to a baseline that only trains with vocalized data, decreasing transcription word error rate from 64% to 4% in one data condition and 88% to 68% in another. To spur further development on this task, we share our new dataset of silent and vocalized facial EMG measurements.')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,authors,arxiv_id,score
0,EasyVoice: Integrating voice synthesis with Skype,Paulo A. Condado and Fernando G. Lobo,0706.3132,0.818029
1,Am\'elioration des Performances des Syst\`emes Automatiques de\n Reconnaissance de la Parole pour la Parole Non Native,"Ghazi Bouselmi (INRIA Lorraine - LORIA), Dominique Fohr (INRIA\n Lorraine - LORIA), Irina Illina (INRIA Lorraine - LORIA), Jean-Paul Haton\n (INRIA Lorraine - LORIA)",0711.1038,0.807498
2,Adjusted Viterbi training for hidden Markov models,"J. Lember, A. Koloydenko",0709.2317,0.770014
3,Exploiting Nonlinear Recurrence and Fractal Scaling Properties for Voice\n Disorder Detection,"Max A Little, Patrick E McSharry, Stephen J Roberts, Declan AE\n Costello and Irene M Moroz",0707.0086,0.766409
4,Probabilistic SVM/GMM Classifier for Speaker-Independent Vowel\n Recognition in Continues Speech,"Mohammad Nazari, Abolghasem Sayadiyan, SeyedMajid Valiollahzadeh",0812.2411,0.761897
5,Acoustic Features and Perceptive Cues of Songs and Dialogues in Whistled\n Speech: Convergences with Sung Speech,Julien Meyer (LAB-Upc),0711.3704,0.760309
6,Combined Acoustic and Pronunciation Modelling for Non-Native Speech\n Recognition,"Ghazi Bouselmi (INRIA Lorraine - LORIA), Dominique Fohr (INRIA\n Lorraine - LORIA), Irina Illina (INRIA Lorraine - LORIA)",0711.0811,0.756685
7,Phoneme recognition in TIMIT with BLSTM-CTC,"Santiago Fern\'andez, Alex Graves, Juergen Schmidhuber",0804.3269,0.753250
8,A biomechanical model of the face including muscles for the prediction\n of deformations during speech production,"Julie Groleau (TIMC), Matthieu Chabanas (TIMC), Christophe Marecaux\n (TIMC), Natacha Payrard, Brice Segaud, Michel Rochette, Pascal Perrier (ICP),\n Yohan Payan (TIMC)",0803.3924,0.750757
9,Suppl\'eance perceptive par \'electro-stimulation linguale embarqu\'ee :\n perspectives pour la pr\'evention des escarres chez le bless\'e m\'edullaire,"Olivier Chenu (TIMC), Nicolas Vuillerme (TIMC), Alexandre\n Moreau-Gaudry (TIMC), Anthony Fleury (TIMC), Jacques Demongeot (TIMC), Yohan\n Payan (TIMC)",0711.3786,0.746030


In [62]:
# This paper was a EMNLP 2020 Honourable Mention Papers
search_papers(title='If beam search is the answer, what was the question?',
              abstract='Quite surprisingly, exact maximum a posteriori (MAP) decoding of neural language generators frequently leads to low-quality results. Rather, most state-of-the-art results on language generation tasks are attained using beam search despite its overwhelmingly high search error rate. This implies that the MAP objective alone does not express the properties we desire in text, which merits the question: if beam search is the answer, what was the question? We frame beam search as the exact solution to a different decoding objective in order to gain insights into why high probability under a model alone may not indicate adequacy. We find that beam search enforces uniform information density in text, a property motivated by cognitive science. We suggest a set of decoding objectives that explicitly enforce this property and find that exact decoding with these objectives alleviates the problems encountered when decoding poorly calibrated language generation models. Additionally, we analyze the text produced using various decoding strategies and see that, in our neural machine translation experiments, the extent to which this property is adhered to strongly correlates with BLEU.')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,authors,arxiv_id,score
0,Infinite Viterbi alignments in the two state hidden Markov models,"J. Lember, A. Koloydenko",0711.0928,0.819104
1,On the Vocabulary of Grammar-Based Codes and the Logical Consistency of\n Texts,{\L}ukasz D\k{e}bowski,0810.3125,0.814937
2,CLAIRLIB Documentation v1.03,"Dragomir Radev, Mark Hodges, Anthony Fader, Mark Joseph, Joshua\n Gerrish, Mark Schaller, Jonathan dePeri, Bryan Gibson",0712.3298,0.806504
3,Questions & Answers for TEI Newcomers,Laurent Romary (LORIA),0812.3563,0.803840
4,The emerging field of language dynamics,S. Wichmann,0801.1415,0.799843
5,In memoriam Maurice Gross,Eric Laporte (IGM-LabInfo),0711.3452,0.799252
6,A constructive proof of the existence of Viterbi processes,"J. Lember, A. Koloydenko",0804.2138,0.798086
7,Morphic and Automatic Words: Maximal Blocks and Diophantine\n Approximation,"Yann Bugeaud, Dalia Krieger, Jeffrey Shallit",0808.2544,0.795205
8,Three Lectures on Automatic Structures,"Bakhadyr Khoussainov, Mia Minnes",0809.3430,0.789193
9,UNL-French deconversion as transfer & generation from an interlingua\n with possible quality enhancement through offline human interaction,"Gilles s\'erasset (IMAG, Clips - Imag, Lig), Christian Boitet (IMAG,\n Clips - Imag, Lig)",0811.0579,0.785540


In [63]:
# This paper was a EMNLP 2020 Honourable Mention Papers
search_papers(title='Spot The Bot: A Robust and Efficient Framework for the Evaluation of Conversational Dialogue Systems',
              abstract='The lack of time efficient and reliable evalu-ation methods is hampering the development of conversational dialogue systems (chat bots). Evaluations that require humans to converse with chat bots are time and cost intensive, put high cognitive demands on the human judges, and tend to yield low quality results. In this work, we introduce Spot The Bot, a cost-efficient and robust evaluation framework that replaces human-bot conversations with conversations between bots. Human judges then only annotate for each entity in a conversation whether they think it is human or not (assuming there are humans participants in these conversations). These annotations then allow us to rank chat bots regarding their ability to mimic conversational behaviour of humans. Since we expect that all bots are eventually recognized as such, we incorporate a metric that measures which chat bot is able to uphold human-like be-havior the longest, i.e.Survival Analysis. This metric has the ability to correlate a bot’s performance to certain of its characteristics (e.g.fluency or sensibleness), yielding interpretable results. The comparably low cost of our frame-work allows for frequent evaluations of chatbots during their evaluation cycle. We empirically validate our claims by applying Spot The Bot to three domains, evaluating several state-of-the-art chat bots, and drawing comparisonsto related work. The framework is released asa ready-to-use tool.')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,authors,arxiv_id,score
0,Goal-oriented Dialog as a Collaborative Subordinated Activity involving\n Collective Acceptance,"Sylvie Saget (IRISA), Marc Guyomard (IRISA)",0805.4101,0.801561
1,CLAIRLIB Documentation v1.03,"Dragomir Radev, Mark Hodges, Anthony Fader, Mark Joseph, Joshua\n Gerrish, Mark Schaller, Jonathan dePeri, Bryan Gibson",0712.3298,0.794038
2,Plate-forme Magicien d'Oz pour l'\'etude de l'apport des ACAs \`a\n l'interaction,"J\'er\^ome Simonin (INRIA Rocquencourt / INRIA Lorraine - LORIA),\n Marius Hategan (INRIA Rocquencourt / INRIA Lorraine - LORIA), No\""elle\n Carbonell (INRIA Rocquencourt / INRIA Lorraine - LORIA)",0708.3740,0.789749
3,"Optimizing Scrip Systems: Efficiency, Crashes, Hoarders, and Altruists","Ian A. Kash, Eric J. Friedman, Joseph Y. Halpern",0705.4110,0.770145
4,"Human-Machine Symbiosis, 50 Years On",Ian Foster,0712.2255,0.767971
5,"The noisy veto-voter model: a Recursive Distributional Equation on [0,1]",Saul Jacka and Marcus Sheehan,0804.3943,0.763506
6,SimDialog: A visual game dialog editor,"C. Owen, F. Biocca, C. Bohil, J. Conley",0804.4885,0.762032
7,Infinite Viterbi alignments in the two state hidden Markov models,"J. Lember, A. Koloydenko",0711.0928,0.760852
8,Comment: Expert Elicitation for Reliable System Design,"Norman Fenton, Martin Neil",0708.0285,0.760090
9,Comment: Expert Elicitation for Reliable System Design,Andrew Koehler,0708.0287,0.760090


In [64]:
# EMNLP 2020 paper on making Sentence-BERT multilingual
search_papers(title='Making Monolingual Sentence Embeddings Multilingual using Knowledge Distillation',
              abstract='We present an easy and efficient method to extend existing sentence embedding models to new languages. This allows to create multilingual versions from previously monolingual models. The training is based on the idea that a translated sentence should be mapped to the same location in the vector space as the original sentence. We use the original (monolingual) model to generate sentence embeddings for the source language and then train a new system on translated sentences to mimic the original model. Compared to other methods for training multilingual sentence embeddings, this approach has several advantages: It is easy to extend existing models with relatively few samples to new languages, it is easier to ensure desired properties for the vector space, and the hardware requirements for training is lower. We demonstrate the effectiveness of our approach for 50+ languages from various language families. Code to extend sentence embeddings models to more than 400 languages is publicly available.')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,authors,arxiv_id,score
0,In memoriam Maurice Gross,Eric Laporte (IGM-LabInfo),0711.3452,0.784882
1,Open architecture for multilingual parallel texts,M.T. Carrasco Benitez,0808.3889,0.771049
2,Bootstrapping Deep Lexical Resources: Resources for Courses,Timothy Baldwin,0709.2401,0.766714
3,Methods to integrate a language model with semantic information for a\n word prediction component,"Tonio Wandmacher, Jean-Yves Antoine",0801.4716,0.752721
4,"Outilex, plate-forme logicielle de traitement de textes \'ecrits","Olivier Blanc (IGM-LabInfo), Matthieu Constant (IGM-LabInfo), Eric\n Laporte (IGM-LabInfo)",0711.3691,0.745125
5,"Commonsense Knowledge, Ontology and Ordinary Language",Walid S. Saba,0808.1211,0.743726
6,International Standard for a Linguistic Annotation Framework,"Laurent Romary (INRIA Lorraine - LORIA), Nancy Ide (INRIA Lorraine -\n LORIA)",0707.3269,0.743712
7,The VO-Neural project: recent developments and some applications,"M. Brescia, S. Cavuoti, G. d'Angelo, R. D'Abrusco, N. Deniskina, M.\n Garofalo, O. Laurino, G. Longo, A. Nocella, B. Skordovski",0806.1006,0.738122
8,Constructing word similarities in Meroitic as an aid to decipherment,Reginald D. Smith,0808.3616,0.737939
9,About the creation of a parallel bilingual corpora of web-publications,D.V. Lande and V.V. Zhygalo,0807.0311,0.736714


In [65]:
# This paper was the EMNLP 2019 Best Paper
search_papers(title='Specializing Word Embeddings (for Parsing) by Information Bottleneck',
              abstract='Pre-trained word embeddings like ELMo and BERT contain rich syntactic and semantic information, resulting in state-of-the-art performance on various tasks. We propose a very fast variational information bottleneck (VIB) method to nonlinearly compress these embeddings, keeping only the information that helps a discriminative parser. We compress each word embedding to either a discrete tag or a continuous vector. In the discrete version, our automatically compressed tags form an alternative tag set: we show experimentally that our tags capture most of the information in traditional POS tag annotations, but our tag sequences can be parsed more accurately at the same level of tag granularity. In the continuous version, we show experimentally that moderately compressing the word embeddings by our method yields a more accurate parser in 8 of 9 languages, unlike simple dimensionality reduction.')

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

,title,authors,arxiv_id,score
0,Methods to integrate a language model with semantic information for a\n word prediction component,"Tonio Wandmacher, Jean-Yves Antoine",0801.4716,0.826351
1,Bit-Optimal Lempel-Ziv compression,"Paolo Ferragina, Igor Nitto and Rossano Venturini",0802.0835,0.798463
2,In memoriam Maurice Gross,Eric Laporte (IGM-LabInfo),0711.3452,0.794045
3,Determining the Unithood of Word Sequences using a Probabilistic\n Approach,"Wilson Wong, Wei Liu, Mohammed Bennamoun",0810.0139,0.786225
4,Bootstrapping Deep Lexical Resources: Resources for Courses,Timothy Baldwin,0709.2401,0.785482
5,Word-Based Text Compression,"Jan Platos, Jiri Dvorsky",0804.3680,0.784538
6,Determining the Unithood of Word Sequences using Mutual Information and\n Independence Measure,"Wilson Wong, Wei Liu, Mohammed Bennamoun",0810.0156,0.782135
7,A Formal Model of Dictionary Structure and Content,"Laurent Romary (INRIA Lorraine - LORIA), Nancy Ide, Adam Kilgarriff",0707.3270,0.779422
8,Pushdown Compression,"Pilar Albert, Elvira Mayordomo, Philippe Moser, Sylvain Perifel",0709.2346,0.777091
9,Trellis-Coded Quantization Based on Maximum-Hamming-Distance Binary\n Codes,Lorenzo Cappellari,0704.1411,0.776343


In [2]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome']

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great']

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
print(embeddings1)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarits
cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

tensor([[ 0.1392,  0.0030,  0.0470,  ...,  0.0641, -0.0163,  0.0636],
        [ 0.0227, -0.0014, -0.0056,  ..., -0.0225,  0.0846, -0.0283],
        [-0.1004, -0.0774, -0.0014,  ..., -0.0010,  0.0718,  0.0221]])
The cat sits outside 		 The dog plays in the garden 		 Score: 0.2838
A man is playing guitar 		 A woman watches TV 		 Score: -0.0327
The new movie is awesome 		 The new movie is so great 		 Score: 0.8939
